# (Semi-)Auto-calibration and threshold setting

This is a brief notebook for checking previous calibrations, 
doing new calibrations (on all or a subset of SiPMs), 
setting thresholds and putting all this into an override file.

See autocalib.ipynb for an extensive example of doing different checks

In [ ]:
%matplotlib inline

import glob
import os
from collections.abc import Callable, Sequence, Iterator, Mapping
from abc import ABC, abstractmethod
from typing import Any

import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.axes import Axes

import numpy as np
import awkward as ak

from lgdo import lh5
from lgdo.lh5.exceptions import LH5DecodeError
from legendmeta import LegendMetadata
from dspeed.processors import get_multi_local_extrema

from sipm_autocalib import *

plt.rcParams["figure.figsize"] = (10, 4)

In [ ]:
# define input (meta)data. 
# specify where DSP files are, where the metadata is, restrict the 
# number of files or select only certain channels

dm = DataManager(
    project_dir = "/mnt/atlas02/projects/legend/sipm_qc",
    dsp_subpath="data/tier/dsp/ssc/p16/r008", 
    #key_selection=["S086"]
    )

In [ ]:
# Load energies. Remember to eliminate the pulser if existing.
energies = get_energies(dm.dsp_files, dm.raw_keys, dm.chmap, take_pulser_from_normal=True)

### Check current config

In [ ]:
# To check if (and for which SiPMs) current calibration/thresholds is still valid:
# Load calibration & thresholds from current state of metadata:
calib, thresholds = read_overrides_from_metadata(dm.metadata_dir, get_timestamp_from_filename(dm.dsp_files[0]))

# Alternative: Load config file(s) by hand; one can manually overwrite:
#calib, thresholds = read_override_file("../out/l200-p13-r002-ath-lar-T%-par_hit-overwrite.yaml")
#calib2, ths2 = read_override_file("../out/l200-p13-r003-anp-lar-T%-par_hit-overwrite.yaml")
#calib = calib | calib2
#thresholds = thresholds | ths2


In [ ]:
# Plot spectra
calibrated_histos = get_calibrated_histograms(energies, calib, (0,5), 200)
_ = plot_all_pe_histograms_and_thresholds(calibrated_histos, thresholds, gridx=True)

### Recalibrate

In [ ]:
# Load a calibration config, containing the metadata needed for a successful calibration
# Copy & adapt a previous file (Jump back here to iterate over changing settings in the config file)

config = load_config_file(os.path.join(dm.project_dir, "sipm-autocalib/config/autocalib-p13-aph-r003.yaml"))

gen_hist_defaults = config["gen_hist_defaults"]
peakfinder_defaults = config["peakfinder_defaults"]
simple_calibration_defaults = config.get("simple_calibration_defaults", {})
advanced_calibration_defaults = config["advanced_calibration_defaults"]
gen_hist_overrides = config.get("gen_hist_overrides", {})
peakfinder_overrides = config.get("peakfinder_overrides", {})
simple_calibration_overrides = config.get("simple_calibration_overrides", {})
advanced_calibration_overrides = config.get("advanced_calibration_overrides", {})

In [ ]:
# Run the simple calibration. TIP: If you want to recalibrate only a subset: either load only subset of channels at
# the start _OR_ crop energies dict down before passing it here. 
simple_calib_output, _, _ = multi_simple_calibration(energies, gen_hist_defaults, peakfinder_defaults, {}, gen_hist_overrides=gen_hist_overrides, peakfinder_overrides=peakfinder_overrides, calibration_overrides=simple_calibration_overrides, draw=True, nodraw_axes=False)

In [ ]:
calibrated_histos = get_calibrated_histograms(energies, simple_calib_output, (0, 5), 200)
_ = plot_all_pe_histograms(calibrated_histos)

In [ ]:
advanced_calib_output, _, _ = multi_advanced_calibration(calibrated_histos, get_calibrated_PE_positions(simple_calib_output), advanced_calibration_defaults, calibration_overrides=advanced_calibration_overrides, draw=True)

In [ ]:
# Do thresholding and plot all newly calibrated spectra for a last check:

adv_calibrated_histos = get_calibrated_histograms(energies, combine_multiple_calibrations(simple_calib_output, advanced_calib_output), (0, 5), 200)
thresholds = multi_valley_minima(adv_calibrated_histos, mimimum_position=0.4)
_ = plot_all_pe_histograms_and_thresholds(adv_calibrated_histos, thresholds, gridx=True)

### Export the final data to an override file

In [ ]:
if True:
    output_override_file("../out/temp_override.yaml", combine_multiple_calibrations(simple_calib_output, advanced_calib_output), thresholds)